## Datastrukturer

Bygger på numpy arrays og arver mye funksjonalitet derfra. Det er to sentrale objekt: Series og DataFrame

I tillegg til den implisitte numeriske indexen til arrays har den også et eksplisitt indexobjekt som mapper til element i Series eller til Series i DataFrame. Koblingen mellom indeks og verdi gjør at vi kan kombinere data fra ulike kilder, håndtere missing data og generelt ikke er avhengig av å ha helt konforme data (samme størrelse, samme rekkefølge) for å gjøre regneoperasjoner.

### Series

Series er crosseover mellom array og dict. Består av to arrays: én index og én med homogene data. Vanlige ndarrays har implisitt index med posisjon, men her er det eksplisitt og kan ha ulike labels. Kan også tenke på Series som en dict som mapper index-verdier til verdi i array. Følgelig er det en del metoder/syntax som tilsvarer dict. Kan få ut hver av arraysene med:
1. a.values (ndarray)
2. a.index (index-objekt, finnes litt ulike typer, har diverse metoder, noe mengdegreier)

### DataFrame

DataFrame er tabulær datastruktur med både rekke-index og kolonne-index. I utgangspunktet er det ganske symmetrisk, men av konvensjon angir rekke-index obseravjon og kolonne-index er variabel (dimensjon/egenskap) ved observasjonen. Kan betrakte det som en dict som mapper label til Series, der labels er enten fra rekke- eller kolonneindex.

### Index

Det er en fullverdi datastruktur/objekt i seg selv. Har en del metoder og sånn. Vet ikke om jeg må jobbe så mye direkte med indexen; kan gjøre operasjon på dataframe og så håndterer de index internt.

Kan bruke eksisterende kolonne som index med df.set_index('kolnavn')

Merk at når vi setter en ny kolonne som indeks mister vi den eksisterende index-kolonnnen. Hvis vi vil beholde må vi lagre kopi av kolonne og deretter putte den inn i nye dataframe. Kan få tak i kolnnen ved å bruke df.index() og deretter dytte den inn i df ved å assigne kolonnen med df["navn"] = df.index()


##### Indeksing

Kan ha fler dimensjonale index, f.eks først by state i kol 1, deretter kommune i kol 2. Begge indexer. Hierarkisk labeling.

Lager ved å gi liste av liste som argument inn i df.setindex

eks: df = df.setindex(["kol1","kol2"])

For å finne gitt rekke må vi gi df.loc["index1","index2"]

Kan endre navn på index ved .rename(columns = {"gammel":"ny}

#### kolonner

Kan merke at kolonnene egentlig bare er index og at det i utgangspuntket er ganske symmetrisk.

Vil ha deskriptive kolonnenavn. Bruker
- df.rename(columns=mapper), der mapper er dictionary med {'gammel':'ny',:}

Tror min konvensjon er å ha navn i snake_case, kan bruke

```python 
def camel_to_snake(s):
    return ''.join('_'+ch.to_lower() if ch.isupper() else ch for ch in s)
df.columns = [camel_to_snake(column) for column in df.columns]

```

    


Kan bruke vanlig list comprehension til å finne subset av kolonner som oppfyller kriterie,

```python
cols_subset = [col for col in df.columns if 'string' in col]
```

#### Hierarkisk index

Kan ha flere nivå på index. Det gjør at vi kan representere data som i utgangspunktet er i høyere dimensjon som tabulær data (eks: panel).

##### Multi index

Finnes eget MultiIndex objekt. Lager ved å sende liste inn i df.setindex()

-  Kan f.eks. få ut df med multiindex hvis vi kjører en groupby på flere kolonner. Aggregerer da på subset av observasjon som har kol1 = A og kol2 = B f.eks. 
-  Kan få ut relativ andel av kategori som har gitt verdi av underkategori ved å lage groupby på multiindex, df.groupby(level=1).sum() .
- Mer at groupby lager delmengder av observasjon med gitt verdi av index. Kan eksplisitt loope over dette:

for idx, gb in groupby(level=1):
    
    df_sub = gb
    
Kan få ut index til hvert nivå med 
df.index.get_level_values(num), der 0 er øverste nivå.

##### Slice multiindex

For å få økt fleksibilitet er liste og tuple ulik tolkning

- df.loc[id11]: all rows where the outer most index value is equal to id11
- df.loc[(id11, id21)]: all rows where the outer-most index value is equal to id11 and the second level is equal to id21
- df.loc[[id11, id12]]: all rows where the outer-most index is either id11 or id12
- df.loc[([id11, id12], [id21, id22]), :]: all rows where the outer-most index is either id11 or id12 AND where the second level index is either id21 or id22
- df.loc[[(id11, id21), (id12, id22)], :]: all rows where the the two hierarchical indices are either (id11, id21) or (id12, id22)

Dersom vi vil begrense oss til noen få indre verdier, men alle ytre, så trenger vi IndexSlice objekt som argument for vår loc

- pd.IndexSlice[:,['A', 'B'],:] gir oss alle ytre, 'A' og 'B' fra nivå innenfor, og verdier fra alle kolonner.

#### Sortering

Kan sortere entent etter index eller verdi langs gitt kolonne,
1. df.sort_index()
2. df.sort_values(by='kolnavn')

## Lage dataframes

### Fra fil

Spesifiserer hvordan pd.readcsv() skal laste inn data

1. usecols = [..] for å spesifisere subset av kolonner vi vil laste inn
2. names = [..] hvis kolonnene ikke har navn fra før så kan vi spesifisere de.
3. Bestemme hvilken kolonne i datasett som skal fungere som index, index_col = <tall>. 
4. Dersom det er rekke vi ikke vil laste inn, skiprows = <tall>, skipper de n øverste.
5. parse_dates = True ---> finner kolonne med date, gjør den til index, får den i datetime.
    
Kan også laste inn fra andre filtyper enn .csv, for eksempel .json. Tar da orient keyword med verdi "split", "records" eller "columns"
    
pd.read_json(orient=)

### Fra datastrukturer

Kan ha lyst til å lage tabell som mapper index til verdi. F.eks. tabell med som mapper variabelnavn til reg koef. Har to lister. Et alternativ er pd.Series(data=x,index=y). Annet alternativ er å først konstruere dict:

pd.Series(dict(zip(y,x))

## Beskrive data

Før jeg kan gjøre noe analyse må jeg først undersøke tabellen med tall jeg har fått utdelt. Må få oversikt over:
1. Hvilke egensakper (kolonner) og hva de måler
2. Hva slags måleenhet de er i og konvertere til riktig datatype. I algebraen under the hood er alt tall, men representer som factor for å få bedre representasjon og utnytte funksjonalitet i plotte-libraries og lignende. Vil unngå å jobbe eksplisitt med dummies før det er nødvendig.
3. Få noe oversikt over univariate fordelinger og eventuelt parvis bivariate korrelasjoner. Se at tall er rimelig, blir kjent med data, oppdage numerisk kodete missing values, se etter feilkodinger
4. Få oversikt over manglende verdier, vurdere strategi for imputation og eventuelt bias dersom vi ser på delutvalg med fullstendig obervasjon.

Kan bruke .describe() --- output avhenger av datatype

Kan bruke .info() til å få datatype og observere om det er missing values

Kan bruke .ndtype() for å finne datatypene

Hvis vi har en series kan vi bruke .value_count() for å få antallet observasjoner med gitt verdi langs den dimensjonen.

Kan bruke .unique() for å få liste av unike verdier og .unique() for antallet unike

### Datatyper

Hva slags type data vi har påvirker gyldige operasjoner og analyse av data. Derfor viktig å kategorisere. Hver kolonne er homogen.

1. Ulike numeriske datatyper (int,float,..)
2. category
3. object

Kan bestemme datatype i series(kolonne) i df ved:

df.kol1.astype('category', ordered=True)

Kan endre datatype til flere kolonner ved : df[liste_av_kolonner] = df[liste_av_kolonner].astype('dtype') 

### Beskrive kolonner

Bruker value_counts(normalize=).sort_index() # normalize hvis jeg vil ha som prosent

gir oversikt over antall unike verdier, hvor tyngden av data ligger og om det er verdier som ikke gir mening.

## Missing values

Vil representere med NaN. Kan lage med np.nan. Dette er en float, slik at kolonner med missing values blir recast til floats. Det er litt teknisk hvordan det blir representert under the hood, får eventuelt se på det senere. I R er det na og null, i pandas bare NaN (finnes en mer moderne datatype na som de har lagt til).

Missing values propapegerer når vi gjør operasjoner på dataframe. Operasjon er elementvis og alt som bruker NaN resultererr i NaN. Dette gjelder også får aggregeringsfunksjoner.

### Filtrering

Første jeg må gjøre er å sjekke antallet missing values i ulike kolonner. Husk at missing values kan være kodet på ulike måter i ulike datasett. En mulighet er å bruke df['col'].unique() til å se på verdier og se om noen ikke gir mening. Deretter vil jeg omkode de til NaN, f.eks. ved df.replace(value,np.nan)

- df.isna() gir boolean mask over hele dataframe
- df.isna().any() angir om hvorvidt det er nans i hver av kolonner
- df.isna().sum() gir antallet nans i hver kolonne

Når jeg har fått oversikt kan jeg vurdere å filtrere de ut. Bruker da df.dropna() med opsjoner
- axis= .. droppe kolonne eller rekke avhengig om jeg finner nans der
- how='any','all'
- tresh= antall nans før jeg dropper

### Imputation

En alternativ fremgangsmåte er å fylle inn verdier. Algoritmer i sklearn kan ikke håndtere nans og det er synd å kaste bort data bare fordi noen observasjoner er litt mangefulle. Finnes ulike strategier for dette som jeg kan se på senere
1. Bruke gjennomsnitt i kolonnen
2. Predikere verdi ut fra andre dimensjoner vi observere
3. Bruke verdi fra naboer

## Data cleaning

Arrays i dataframe er homogone. Dersom det er noen symboler i kolonne som i utgangspunktet skal være numerisk vil pandas tolke det som string. Må fjerne symbolene før vi kan konvertere.
```python
def to_numeric(s):
    return int(''.join(ch for ch in s if ch.isnumeric()))
```

## Split-apply-combine workflow

Vi har et stort datasett. Det er mange variabler og mange kolonner. Hvordan skal vi få ut noen innsikter fra all denne informasjonen?

Kan vært veldig nyttig å dele det inn i mindre deler etter felles verdi langs kolonne. Kan tenke at det er observasjoner som "hører sammen". Deretter annvender vi noen aggregeringsfunksjoner som gir oss slags sammendragsmål for tallverdiene i hver del. Deretter kan vi kombinere det sammen i nytt datasett som mapper nøkkel (felles verdi) til verdi av aggregeringsfunksjon.

Okay, la oss se på dette i praksis:

gbA = df.groupby('A') gir oss et groupby objekt. Det er poeng at hver gruppe er en dataframe. Kan få ut disse dataframene med gbA.get_group(verdi av A). Kan gruppere med verdier langs flere kolonner; dette gir og PxK grupper, der P er verdi langs første og K er verdi langs andre. I den kombinere dataframen vil har multiindex. Kan også observere at hver observasjon er i én gruppe.

Kan anvende aggregeringsmetoder direkte på gbA. Kan også definere custom aggregeringfunksjon og anvende de med .agg metoden

gbA.agg(lambda x: (x > 0).count()) gir antaller positive observasjon. Kan være greit å gi funksjonene navn så blir litt mer ryddig.

For å øke fleksibiliteten til vår oppdeling av datasett kan vi lage et Grouper objekt. Stor fordel her Grouperen vår kan tolke datetime objekter. Hvis key=series med datetime så kan vi slenge in freq="W" og gruppere observasjoner for hver uke

eksempel: gb = df.groupby(pd.Grouper(key='Date', freq ='W')) der Date er kolonne med datetime objekt.

### .agg() metoden

Hvis vi har en groupby objekt og vil finne aggegert tallmål på verdiene i annen kolonne assosiert med hver verdi i groupby

eks: df.groupby("Kol1").agg({"kol2": np.mean}) ...

### Apply og applymap

Kan ha lyst til å lage custom funksjoner gir ett output per kolonne/rekke. Definerer egen funksjon, f = lambda x: .., der x er series, og tar deretter df.apply(f)

Ofte vil jeg lage en kolonne der verdi avhenger verdi observasjon har langs annen kolonne. Påfører funksjon elemntvis, men kan vektorisere med map. 

eks: df['ny_col']=df['col'].map(lambda x: 2*x)

F.eks hvis vi vil generere en variabel som er en funksjon av verdiene av to korresponderende verdier i to iterables

f.eks: x3i = x1i*x2i , i= 1, ... , n

x3 = map(lambda x,y:x*y,x1,x2)


### Pivot tables

Lage en df fra eksisterende df for å analysere spesifikk problem. Bestemmer index. Lar verdiene i én av kolonnene være kolonner(kategorier) i ny df. Bestemmer hvilken kategori som skal være verdi for hver index langs nye kolonner. Spesifisere aggregeringsfunksjon for alle verdiene for gitt katagori for hver index (flere observasjoner inn i samme rute --> aggregering til ett tall).

eks: df.pivot_table(values=BNP , index=ÅR ,columns =LAND, agg=np.mean) 

## Reshape


Vi kan representerer de samme dataene på ulike måter. 

1. Long (alt er i index...)
2. Wide (alt er i kolonne...)

Vi har fire grunnoperasjon for å omforme data

1. stack (sender ting ned i index, flere levels på indexen)
2. unstack (sender ting opp i kolonne, mindre levels på indexen)
3. set_index
4. reset_index

Har operasjoner som bruker disse under the hood

1. melt (gjør long)
2. pivot (index, col, value) <- wide?
3. pivot_table, generalisering som gjør at vi kan spesifisere agg funksjon dersom flere verdier til (index, col) par

Eksempel på melt.. Hvis jeg har mange kolonner med verdier, eks: uke 1, uke 2, uke 3..... vil heller ha key value (en kolonne med uke, en kolonne med verdien den uke). Bare spesifiser at alle andre kolonner er id-vars

Kan ha tabell i stacked format. egen kolonne som hva slags type variabel og deretter kolonne med verdier,

|index|type |verdi |
|---|---|---|
|1|a|1|
|1|b|2|
|2|a|3|

for å få dette på tidy format der innehold i hver celle korresponderer med et (key,type) par så må jeg pivote

df.pivot(index=index,columns=type,values=verdi)

Tenker at index + columns må identifisere unik observasjon, deretter bruke values kolonne til å fylle inn verdier i columns

## Kombinere dataframes

Har fire typer joins som bestemmer hvilke observasjoner som blir med i det nye datasettet.
1. inner
2. left
3. right
4. outer

for de tre nederste blir observasjon som ikke har verdi i en kolonne paddet med nans

### concat

Hvis har samme kolonner eller samme index i to dataframes så er det enklest å bare slenge de sammen med

pd.concat([liste av dfs],axis=)

padder på med nans avhengig av hvilke type join. Beholder indexverdi.

### merge

Bruker verdi i felles kolonner til å merge verdier. Tenk at vi har observasjon om hvilke land individer kommer fra. På bakgrunn av dette vil vi koble på mer informasjon om landet på hvert individ. Vi har et annet datasett med informasjon om hvert land. Kan da koble dette på våre individdata med land som 'key' kolonne. Den driter i

pd.merge(left,right) tilsvarer left.merge(right)

left.merge(right,on='key',how=) 

- Ulike navn på kolonneindex som merger på: left_on=, right_on=
- Hvis kolonnen er plassert som index så bruker vi bare left_index=True, right_index=True. Hvis indexen har navn så kunne vi brukt left_on=

Hvis vi vil merge to datasett med samme type index (ie. liste av navn, land .. whatever) så bruker vi pd.merger(a,b,options). Siden det ofte vil være slik at noen index ikke har verdier for noen av kolonnene har vi masse options for hvordan merge.

1) how = left -- kun obs som har verdier til kol a

2) how = right -- tilsvarende

3) how = inner (snitt) -- verdier på begge

4) how = outer (union) -- verdier på minst én

Vi må også spesifisere hva som er indexene for mergingen. left_on, right_on, left_index=True etc|

### join

Convience funksjon som bruke pd.merge under the hood, men kan være bedre valg dersom vi bruker index til å koble data

### Legge til series (rad og kolonne)

Hvis vi bruker df.append() så lages nytt objekt. Må sende en series eller dataframe som argument. 

Eks: df = dt.append(pd.Series({dict: keys:values}), der keys korresponderer med kolonne indexer 


## Querying dataframe 

Kan bruke bools til å indexe. Gir ut array som "bli lagt oppå dataframen". Får ut de verdiene som korresponderer med True. 

Boolean masking er viktig!! Apply operatorer til dataframe.

1) Lage boolean array, f.eks: df[<column name>]<condition>. Får tilgang til kolonne og tester hver verdi opp mot condition. Returner array med samme størrelse der verdiene er enten true/false.

2) "Apply the mask to the dataframe". Lager nå df, df_new = df.where(<mask>). Altså bruke where method og boolsk array som input. Ny df har samme størrelse som gamle, men data fra rows hvor betingelse = false ---> NaN verdier. For å droppe disse rekke kan vi bruke df = df.dropna()
    

Kan gjøre begge deler i ett steg: df_new = df[df[<kolonne>]<condition>]] Bruke boolsk array som index til original df. Jævla clean og gjør enkelt å lage mer kompliserte logiske operasjon ved å binde sammen med | (eller) & (og). Der hver del er innrammet i parantes.

Eksempel: df_new = df[(df[["gold"] > 0) & (df["gold1"] == 0])]

Eksempel 2: df_new = df[<col>][<bool mask>]. Gir verdiene fra kolonne der bool mask (som kanskje kommer fra annen kolonne i df) har verdi True. 

Eksempel 3: df_new = df[df.NavnPåKolonne == verdi]

## Dato-funksjonaliteter i pandas

Det er to hoved-classer i pandas for å behandle tid-data

1) Timestamp --- spesifikk tidspunkt

2) Period -- for intervall av tidspunkt

i tillegg egen klasse for tid som indeks i series.

Bruker .to_datetime() for å konvertere liste med representasjoner av tid til datetime.

Vanligvis har vi tiden oppgitt som string, må legge inn "kode" for hvordan den konverteres til datetime som argument

eks: format = '%Y%m%d'

Tror dette bygger på datetime library som har i hvertfall to classer:

1) datetime

2) timedelta, er differanse mellom datetime objekt, greit når vi skal gjøre aritmetikk

har masse metoder som gjør de enklere å regne med. har mange flere funksjonaliteter enn en string

## Diverse data cleaning 

Fjerne sjit fra en liste av strings: bruker list comprehension for å generere en ny liste der hver string erstattes med string.replace("","")

eks : a = [i.replace("-","") for i in liste] # litt problem at vi mister index.. av diverse årsaker vil jeg alltid beholde index i stedet for å få en ren array

Annen metode er å bruke slicing dersom det er mønster i dataene vi kan utnytte

Hvis vi vil fjerne alle symboler som ikke er tall fra strings kan vi bruker

df['name'] = df['name'].map(lambda x: ''.join([ch for ch in x if ch.isdigit()]))



## String methods

Hvis kolonne er string kan jeg bruke df.col.str for å få tilgang til metoder. Dette er ofte bedre å raskere enn å gjøre en eksplisitt loop og bruke string metode på innholdet i hver celle. Har de fleste (alle?) metodene som native python har på strings. Har i tillegg noen egne funskjonaliteter knyttet til pandas, feks:

- df['col'].str.get_dummies(sep='|'), lage dummmies der hver observasjon kan inneholde flere verdier i form av string

Hvis jeg vil bruke flere str operasjoner i chain må jeg bruke .str etter hver, eks
- data['Min_Salary']=data['Min_Salary'].str.strip(' ').str.lstrip('$').str.rstrip('K').fillna(0).astype('int')

a.split('x',maxsplits=n), splitter på de n første tilfellene av 'x'. Fjerner symbolet som det splittes på. Hvis jeg vil ha ut string innhold før split bruker jeg a.split('x',maxsplits=n).str[0] ...


## Generelle tips

Hvis vi jobber med stor dataframe og kun er interessert i liten del for å svare på et gitt spørsmål, så kan det være en god idé å lage en ny dataframe som er subset av den opprinnelige. 

Stiltips:

1) Unngå ][ , chain indexing

2) Chain metoder. En måte å gjøre dette mer oversiktelig er å fordele det over flere linjer (analog til dplyr), eks:

```python
(df.groupby["NN"]
     .MM # velge kolonne
    .dropna() # kommentar
    .mean() # kommentar
    )
```

## Annet

### pd.cut()

Når vi vil dele kontinuerlige data inn i bins. Konstruere 'factor' med levels, kan deretter lage dummies

pd.cut(x=kolonne,,right=True,bins=,labels=,..)

bins kan ta ulike form:
1. int, spesifiserer antall bins, partisjonerer (min(x),max(x)) inn i like store intervall
2. sekvens av tall (a,b,c,..), lager intervall (a,b],(b,c] hvis right=True. Gir NaN hvis ikke i angitte intervall.
3. IntervalIndex (vet ikke)

som default blir output en kolonne som viser hvilket intervall observasjon tilhører. Kan gi labels som må være iterable med samme lengde som antall bins for å gi annen representasjon.

- pd.cut(series, bins) gir oss en series med index til series og verdier indelt i et gitt antall bins. Blir litt som et histogram. Deler intervallet inn i like store deler. Med default får vi ut hvilke intervall verdien av variabelen er innenfor. 
Hvis vi setter labels=False får vi ut hvilket posisjon den har i rekkefølge av intervaller.
- pd.qcut( ) har tilsvarende atferd som over, men i stedet for å dele intervallet i like store delintervall så deler den det i intervall der det er like mange observasjoner innenfor hver.


Kan lage lagged kolonner med .shift()

Hvis jeg vil summe alle elementene i en dataframe så er det bare å kjøre df.sum().sum(), første gir series, andre summer opp series.

### Andre typer objekt i pandas

Vi kjenner til groupby-objekt. Finnes også andre objekt som vi kan utføre aggregeringsfunksjoner på og få ut Series eller df.

-  a = pd.Series(..), a.rolling(window=antall obs) <- gir oss et rolling objekt med en rekke metoder.
-  a.mean() gir oss en Series med rullende gjennomsnitt. Denne seriesen kan vi f.eks. bruke til å plotte.
-  Et alternativ til aritmetisk gjennomsnitt er at tyngden til observasjon er vektet etter tid. Kan bruke a.ewm() til å få eksponentielt vektet ting.

### Nyttige metoder

-  a.pct_change() gir oss en ny series (eller df hvis flere kolonner) der hver observasjon fra i=1->n er erstattet med df[i]/df[i-1]-1
- Kan vi bruke dette til å finne aggregert avkastning? Ja. Men hvordan?? hmm???
- a.nlargest() gir series med index og verdi til n største verdiene i en series
- a.transform(func) gir en series med samme størrelse der vi har kjørt en funksjon på den.. Kan også brukes på dataframes. Uansett, eksempel er b = a.transform(lambda x: pd.cut(x, 100))
- a.str gir oss tilgang til mange string metoder som vi kan anvende på alle elementer i series. raskere enn å kjøre det i en loop.
- df.iterrows() gir oss et generatorobjekt som vi kan bruke til å loope oss gjennom df row by row. Hvert element er en tuple med rowindex og rowen som series. Følger da at index til min row series er kolonnenanvnene. 
- agg og apply, vet ikke helt hva som er forskjellen
- stack gjøre kolonnenavn til indexverdier
- unstack gjør indexverdier til kolonnenavn
- reindex til å velge subset av index/kolonne når jeg har liste av tall jeg vil subsette med
- idxmax() gir første index til høyeste verdi langs gitt index.. hvis axis=1 så får vi kolonnen med høyest verdi 

### Nyttige funksjoner

- ufuncs fungerer også på dataframe
- Har innebygde funksjoner som gir én output per kolonne/rekke (typ statistiske/matematiske funksjoner)
- pd.get_dummies() lager dummy dataframe fra kategorisk variabel. Kan spesifisere predix="string" slik at de får kolonnenavn ["string_0","string_1,...]

In [4]:
import pandas as pd
import numpy as np

In [ ]:
np.var()

In [3]:
df = pd.DataFrame()

In [ ]:
df.idxmax()